## Decorator Anatomy

- Outer decorator function accepts the target function and creates a wrapper inside it.
- The wrapper usually takes *args, **kwargs so it can handle any signature.
- Wrapper executes optional "before" code, calls the original, maybe does "after" code, and returns the original’s result.
- Returning the wrapper from the decorator completes the transformation.


Using decorators:

- Manually wrapping illustrates what @ syntax really does behind the scenes.
- This approach is clear but repetitive: @ eliminates the manual reassignment step.

1. *args (Đối số vị trí tùy ý)

Khi bạn đặt *args trong định nghĩa hàm, nó sẽ gom tất cả các đối số vị trí được truyền vào thành một tuple.

Ví dụ đơn giản: Hàm tính tổng

Hãy tưởng tượng bạn muốn viết một hàm tinh_tong có thể cộng 2, 3, 4, hay bao nhiêu số tùy ý.

In [6]:
def tinh_tong(*args):
    print(f"Các đối số được gom vào một tuple: {args}")
    print(f"Kiểu dữ liệu của args là: {type(args)}")
    
    tong = 0
    for so in args:
        tong += so
    return tong

# Gọi hàm với số lượng đối số khác nhau
print("Tổng của 1, 2 là:", tinh_tong(1, 2))
print("Tổng của 1, 2, 3, 4, 5 là:", tinh_tong(1, 2, 3, 4, 5))

Các đối số được gom vào một tuple: (1, 2)
Kiểu dữ liệu của args là: <class 'tuple'>
Tổng của 1, 2 là: 3
Các đối số được gom vào một tuple: (1, 2, 3, 4, 5)
Kiểu dữ liệu của args là: <class 'tuple'>
Tổng của 1, 2, 3, 4, 5 là: 15


2. **kwargs (Đối số từ khóa tùy ý)

Tương tự, **kwargs sẽ gom tất cả các đối số từ khóa (có dạng key=value) mà bạn truyền vào thành một dictionary.

In [1]:
def hien_thi_thong_tin(**kwargs):
    print(f"Các đối số được gom vào một dictionary: {kwargs}")
    print(f"Kiểu dữ liệu của kwargs là: {type(kwargs)}")
    
    for key, value in kwargs.items():
        print(f"- {key.replace('_', ' ').title()}: {value}")

# Gọi hàm với các bộ key-value khác nhau
print("Thông tin người dùng:")
hien_thi_thong_tin(ten="Alice", tuoi=30, thanh_pho="Hanoi")
print("-" * 20)
print("Thông tin sản phẩm:")
hien_thi_thong_tin(ten_san_pham="Laptop Dell XPS", gia=45000000, con_hang=True)

Thông tin người dùng:
Các đối số được gom vào một dictionary: {'ten': 'Alice', 'tuoi': 30, 'thanh_pho': 'Hanoi'}
Kiểu dữ liệu của kwargs là: <class 'dict'>
- Ten: Alice
- Tuoi: 30
- Thanh Pho: Hanoi
--------------------
Thông tin sản phẩm:
Các đối số được gom vào một dictionary: {'ten_san_pham': 'Laptop Dell XPS', 'gia': 45000000, 'con_hang': True}
Kiểu dữ liệu của kwargs là: <class 'dict'>
- Ten San Pham: Laptop Dell XPS
- Gia: 45000000
- Con Hang: True


In [1]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with arguments: {args}")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def add(a, b):
    return a + b

print(add(3, 4))


Calling add with arguments: (3, 4)
7


## Decorators with Arguments

- A basic decorator adds fixed behavior; sometimes you need to configure that behaviour (e.g. how many retries, which log level).
- You cannot pass options directly to a plain @decorator, because that decorator receives only the target function.
- Solution: call a factory that takes options and returns a decorator, then apply it with @factory(option=value).

In [5]:
import random

def retry(max_attempts=3):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_attempts + 1):
                try:
                    print(f"Attempt {attempt}/{max_attempts}")
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f" Error: {e}")
                    if attempt == max_attempts:
                        raise

        return wrapper
    return decorator

@retry(4)
def sometimes_fails():
    if random.random() < 0.7:
        raise RuntimeError("Flaky failure")
    return "Success!"

print(f"Result: {sometimes_fails()}")

Attempt 1/4
 Error: Flaky failure
Attempt 2/4
Result: Success!


## Decorators & Return Values

- A decorator’s wrapper replaces the original function, so if it forgets to return the original result the caller receives None
- Fixing this means capturing the result of func(*args, **kwargs) inside the wrapper and returning it unchanged

In [ ]:
def log_calls_broken(func):
    def wrapper(*args, **kwargs):
        print(f"LOG: Calling {func.__name__}")
        func(*args, **kwargs)
        print(f"LOG: Finished {func.__name__}")
    return wrapper

@log_calls_broken   # This is equivalent to:
def add(x, y):      # add = log_calls_broken(add)
    return x + y

print(f"Result seen by caller: {add(2, 3)}")

LOG: Calling add
LOG: Finished add
Result seen by caller: None
